<a href="https://colab.research.google.com/github/taylorvroman09/Taylor-Public/blob/main/Day24_f21_FantasyCharacterNameGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fantasy Character Name Generator

If you haven't done the web-scraping tutorial/demonstration yet, STOP. 

Go back and do that so that you have the correct file. 

In this code we'll build a Character Name Generator [from the file that we webscraped here](https://github.com/merriekay/CS167Code/blob/main/Day24_f21_WebScrapingDemo.ipynb) 


In [1]:
#imports and things
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    !pip install -q -U tensorflow-addons
    !pip install -q -U transformers
    IS_COLAB = True
except Exception:
    IS_COLAB = False

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. LSTMs and CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

     |████████████████████████████████| 1.1 MB 5.5 MB/s 
     |████████████████████████████████| 3.1 MB 5.1 MB/s 
     |████████████████████████████████| 61 kB 429 kB/s 
     |████████████████████████████████| 3.3 MB 33.4 MB/s 
     |████████████████████████████████| 596 kB 43.2 MB/s 
     |████████████████████████████████| 895 kB 39.9 MB/s 


# Import the Data
Import the data (it's a text file so it's a little bit different than a csv). 
- Then we get rid of the newlines and replace it with a space.
- we then get the vocabulary, and use a tokenizer to convert the text to sequences. 

In [3]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
#names = pd.read_csv('/content/drive/MyDrive/sanderson_names.txt',  header = None)
#names.head()

with open('/content/drive/MyDrive/MachineLearning/sanderson_names.txt') as f:
    names = f.read()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
names = names.replace('\n'," ")
names[:100]

'Aarik Aaron Abaray Abiajan Abigail Reed Abraham Desjardins Abrial Abrobadar Abronai Abry Absence Aci'

In [5]:
# The vocabulary of our character-level language model looks like this:
"".join(sorted(set(names.lower())))

" '()-./:abcdefghijklmnopqrstuvwxyz©±√"

In [6]:
# Use Tokenizer to tokenize the Names
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(names)

In [7]:
# Embed the name 'Shallan' as tokens:
tokenizer.texts_to_sequences(["Shallan"])

[[8, 11, 2, 7, 7, 2, 5]]

In [8]:
# Revert the sequence of tokens back to the word:
tokenizer.sequences_to_texts([[8, 11, 2, 7, 7, 2, 5]])

['s h a l l a n']

In [9]:
max_id = len(tokenizer.word_index) # number of distinct characters
dataset_size = tokenizer.document_count # total number of characters

[encoded] = np.array(tokenizer.texts_to_sequences([names])) - 1
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

n_steps = 100
window_length = n_steps + 1 # target = input shifted 1 character ahead
dataset = dataset.repeat().window(window_length, shift=1, drop_remainder=True)

dataset = dataset.flat_map(lambda window: window.batch(window_length))

np.random.seed(42)
tf.random.set_seed(42)

batch_size = 32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows: (windows[:, :-1], windows[:, 1:]))

dataset = dataset.map(
    lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))

dataset = dataset.prefetch(1)


for X_batch, Y_batch in dataset.take(1):
    print(X_batch.shape, Y_batch.shape)

(32, 100, 37) (32, 100)


# Build our model
Now we build our model. 

In [10]:
def create_model():
  model = keras.models.Sequential([
      keras.layers.GRU(64, return_sequences=True, input_shape=[None, max_id],
                      dropout=0.2),
      keras.layers.GRU(64, return_sequences=True,
                      dropout=0.2),
      keras.layers.TimeDistributed(keras.layers.Dense(max_id,
                                                      activation="softmax"))
  ])
  return model


model = create_model()
print(model.summary())

checkpoint_path = "training_1/fantasy_name_gen.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

# Now let's train our model. Notice the callbacks=[cp_callback], 
#this will save checkpoints so we can load our model later.
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam")
history = model.fit(dataset, steps_per_epoch=train_size // batch_size,
                    epochs=10,callbacks=[cp_callback])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, None, 64)          19776     
                                                                 
 gru_1 (GRU)                 (None, None, 64)          24960     
                                                                 
 time_distributed (TimeDistr  (None, None, 37)         2405      
 ibuted)                                                         
                                                                 
Total params: 47,141
Trainable params: 47,141
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
492/492 [==============================] - ETA: 0s - loss: 2.6599
Epoch 00001: saving model to training_1/fantasy_name_gen.ckpt
492/492 [==============================] - 31s 44ms/step - loss: 2.6599
Epoch 2/10
492/492 [==============

# Now let's test some input:


In [11]:
def preprocess(texts):
    X = np.array(tokenizer.texts_to_sequences(texts)) - 1
    return tf.one_hot(X, max_id)

# Let's pass in 'Meredi' and see what it predicts the next letter should be according to Sanderson:
X_new = preprocess(["Meredi"])

#this line takes a look at the softmax output and returns the max
Y_pred = np.argmax(model(X_new), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1] # 1st sentence, last char

'n'

In [12]:
def next_char(text, temperature=1):
    X_new = preprocess([text])
    y_proba = model(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [18]:
tf.random.set_seed(42)

next_char("Meredi", temperature=1)

's'

In [19]:
def complete_text(text, n_chars=50, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [20]:
tf.random.set_seed(42)

print(complete_text("t", temperature=0.3))

tin lishin lishir listin liston listin leshin lesha


In [21]:
print(complete_text("t", temperature=1))

tingbran mushna mortra moral morle mralinsmoshis ca


In [22]:
print(complete_text("t", temperature=2))


tpari uau duqqa d'furc litdywapi qem lim tlvidanaau


In [53]:
import random
new_name = complete_text('Taylo', 15, temperature=0.75)
new_name.split(" ")[0].title()

'Taylord'

# Let's try loading our model

In [ ]:
!pip install pyyaml h5py

In [ ]:
os.listdir(checkpoint_dir)

['fantasy_name_gen.ckpt.data-00000-of-00001',
 'checkpoint',
 'fantasy_name_gen.ckpt.index']

In [ ]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_1/fantasy_name_gen.ckpt'

In [ ]:
# disable warnings becuase we live dangerously:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Create a new model instance
model = create_model()

# Load the previously saved weights
model.load_weights(latest)

print(complete_text("t", temperature=0.25))

ther stend strang strave sterien starfalls starfall


# Next step: Deploy model

This is a bit outside of the scope of this course, but here's an example

Deploy the model using Flask: https://github.com/mtobeiyf/keras-flask-deploy-webapp
